 # Konfiguracja notatnika

In [1]:
import pandas as pd
import requests
import json

from datetime import datetime
from dateutil.relativedelta import relativedelta
from time import sleep

In [2]:
endpoint_url = 'https://dev-apidatalab.coderslab.pl/api/v2'
token = '$2a$12$Fi54uYaGf1KK20k01NnSLeCrCrk9RfSIgO2j0WIQs6s4s7YmTGHl6'
request_offset = 0.3

authorization = {'authorization': token}

In [3]:
airport_list = pd.read_csv('../data/airports.csv')
airports = airport_list['origin_airport_id'].values.tolist()

 # Pobieranie `Airport`

In [4]:
airport_endpoint_url = f"{endpoint_url}/airport"

In [5]:
airport_data = []
for airport_id in airports:
    sleep(request_offset)

    r = requests.get(
        f"{airport_endpoint_url}/{airport_id}",
        headers=authorization,
    )

    if r.status_code != 200:
        continue

    response = json.loads(r.text)
    airport_data.append(response)

In [6]:
airport_df = pd.DataFrame.from_records(airport_data)
airport_df.head()

ORIGIN_AIRPORT_ID            DISPLAY_AIRPORT_NAME ORIGIN_CITY_NAME  \
0              11638             Fresno Air Terminal       Fresno, CA   
1              13342          General Mitchell Field    Milwaukee, WI   
2              13244           Memphis International      Memphis, TN   
3              15096  Syracuse Hancock International     Syracuse, NY   
4              10397               Atlanta Municipal      Atlanta, GA   

                                                NAME  
0               FRESNO YOSEMITE INTERNATIONAL, CA US  
1                  MILWAUKEE MITCHELL AIRPORT, WI US  
2               MEMPHIS INTERNATIONAL AIRPORT, TN US  
3      SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US  
4  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...

 ## Sprawdzenie

In [7]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

In [8]:
airport_df.to_csv(
    '../data/raw/airport_list.csv', 
    sep=';',
    index=False)

 # Pobieranie `Weather`


In [9]:
months_amount = 15
start_date = datetime(2019, 1, 1)
airport_weather_endpoint_url = f"{endpoint_url}/airportWeather"

In [10]:
def get_weather_data(date):
    r = requests.get(
        airport_weather_endpoint_url,
        headers=authorization,
        params={'date': date.strftime('%Y-%m')}
    )

    print(date.strftime('%Y-%m'))
    response = json.loads(r.text)

    return response

In [11]:
airport_weather_data = []

for month in range(months_amount):
    print(f"Pobieram {month+1} z {months_amount}")
    date = start_date + relativedelta(months=month)
    daily_weather = get_weather_data(date)
    print(len(daily_weather))

    airport_weather_data += daily_weather
    sleep(request_offset)

Pobieram 1 z 15
2019-01
3286
Pobieram 2 z 15
2019-02
2964
Pobieram 3 z 15
2019-03
3282
Pobieram 4 z 15
2019-04
3180
Pobieram 5 z 15
2019-05
3285
Pobieram 6 z 15
2019-06
3180
Pobieram 7 z 15
2019-07
3285
Pobieram 8 z 15
2019-08
3286
Pobieram 9 z 15
2019-09
3179
Pobieram 10 z 15
2019-10
3286
Pobieram 11 z 15
2019-11
3176
Pobieram 12 z 15
2019-12
3286
Pobieram 13 z 15
2020-01
2571
Pobieram 14 z 15
2020-02
2407
Pobieram 15 z 15
2020-03
2573


In [12]:
airport_weather_df = pd.DataFrame.from_records(airport_weather_data)
airport_weather_df.head()

WT18      STATION                                               NAME  \
0  None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
1  None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
2  None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
3  None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
4  None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   

         DATE   AWND  PRCP  SNOW  SNWD  TAVG  TMAX  ...  PGTM  WT10  WESD  \
0  2019-01-01   4.70  0.14   0.0   0.0  64.0  66.0  ...   NaN   NaN   NaN   
1  2019-01-02   4.92  0.57   0.0   0.0  56.0  59.0  ...   NaN   NaN   NaN   
2  2019-01-03   5.37  0.15   0.0   0.0  52.0  55.0  ...   NaN   NaN   NaN   
3  2019-01-04  12.08  1.44   0.0   0.0  56.0  66.0  ...   NaN   NaN   NaN   
4  2019-01-05  13.42  0.00   0.0   0.0  49.0  59.0  ...   NaN   NaN   NaN   

   SN32  SX32  PSUN  TSUN  TOBS  WT07  WT11  
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 33 columns]

 ## Sprawdzenie


In [13]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku


In [14]:
airport_weather_df.to_csv(
    '../data/raw/airport_weather.csv', 
    sep=';',
    index=False)

 # Pobranie `Aircraft`


In [15]:
aircraft_endpoint_url = f"{endpoint_url}/aircraft"
r = requests.get(aircraft_endpoint_url, headers=authorization)
data = json.loads(r.text)

In [16]:
aircraft_df = pd.DataFrame.from_records(data)
aircraft_df.head()

MANUFACTURE_YEAR TAIL_NUM  NUMBER_OF_SEATS
0              1944   N54514              0.0
1              1945   N1651M              0.0
2              1953   N100CE              0.0
3              1953   N141FL              0.0
4              1953   N151FL              0.0

 ## Sprawdzenie


In [17]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku


In [18]:
aircraft_df.to_csv(
    '../data/raw/aircraft.csv', 
    sep=';',
    index=False)

 # Pobranie `Flight`


In [19]:
flight_endpoint_url = f"{endpoint_url}/flight"

In [22]:
def get_airport_data(airport_id):
    result = []
    start_date = datetime(2019, 1, 1)

    for month in range(months_amount):
        date = start_date + relativedelta(months=month)
        print(f"\t{date}")

        try:
            response = get_airport_day_data(airport_id, date)
            result += response

            sleep(request_offset)
        except ValueError as e:
            print(f"No data available for airport {airport_id}")
            break

    return result


def get_airport_day_data(airport_id, date):
    payload = {
        'airportId': airport_id,
        'date': date.strftime('%Y-%m')
    }

    r = requests.get(
        flight_endpoint_url,
        headers=authorization,
        params=payload)

    if r.status_code == 204:
        raise ValueError("No data available")

    response = json.loads(r.text)
    return response

In [23]:
flight = []
airports_to_download = airport_df['ORIGIN_AIRPORT_ID'].to_list()
airports_amount = len(airports)

for idx, airport in enumerate(airports):
    print(f"{airport}: {idx} of {airports_amount}")
    airport_flight = get_airport_data(airport)
    flight += airport_flight

10874: 0 of 364
	2019-01-01 00:00:00
No data available for airport 10874
11233: 1 of 364
	2019-01-01 00:00:00
No data available for airport 11233
13360: 2 of 364
	2019-01-01 00:00:00
No data available for airport 13360
15008: 3 of 364
	2019-01-01 00:00:00
No data available for airport 15008
11638: 4 of 364
	2019-01-01 00:00:00
No data available for airport 11638
14150: 5 of 364
	2019-01-01 00:00:00
No data available for airport 14150
15323: 6 of 364
	2019-01-01 00:00:00
No data available for airport 15323
14814: 7 of 364
	2019-01-01 00:00:00
No data available for airport 14814
12007: 8 of 364
	2019-01-01 00:00:00
No data available for airport 12007
11337: 9 of 364
	2019-01-01 00:00:00
No data available for airport 11337
13342: 10 of 364
	2019-01-01 00:00:00
No data available for airport 13342
15070: 11 of 364
	2019-01-01 00:00:00
No data available for airport 15070
13244: 12 of 364
	2019-01-01 00:00:00
No data available for airport 13244
12280: 13 of 364
	2019-01-01 00:00:00
No data av

In [25]:
flight_df = pd.DataFrame.from_records(flight)
flight_df.head()

MONTH  DAY_OF_MONTH  DAY_OF_WEEK OP_UNIQUE_CARRIER TAIL_NUM  \
0      1            20            7                WN   N204WN   
1      1            20            7                WN   N8682B   
2      1            20            7                WN   N717SA   
3      1            20            7                WN   N709SW   
4      1            20            7                WN   N7864B   

   OP_CARRIER_FL_NUM  ORIGIN_AIRPORT_ID  DEST_AIRPORT_ID  CRS_DEP_TIME  \
0                682              10397            11292           605   
1               2622              10397            11292          2120   
2               2939              10397            11292          1800   
3               3848              10397            11292          1355   
4               1352              10397            11697          1125   

   DEP_TIME  ...  CRS_ELAPSED_TIME ACTUAL_ELAPSED_TIME  DISTANCE  \
0     602.0  ...               205               204.0      1199   
1    2114.0  ...               210               205.0      1199   
2    1807.0  ...               210               220.0      1199   
3    1354.0  ...               205               204.0      1199   
4    1125.0  ...               120               124.0       581   

   DISTANCE_GROUP  YEAR CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  \
0               5  2019           NaN            NaN        NaN   
1               5  2019           NaN            NaN        NaN   
2               5  2019           4.0            0.0       10.0   
3               5  2019           NaN            NaN        NaN   
4               3  2019           NaN            NaN        NaN   

   SECURITY_DELAY  LATE_AIRCRAFT_DELAY  
0             NaN                  NaN  
1             NaN                  NaN  
2             0.0                  3.0  
3             NaN                  NaN  
4             NaN                  NaN  

[5 rows x 27 columns]

 ## Sprawdzenie


In [26]:
flight_df_expected_shape = (1386120, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku

In [27]:
flight_df.to_csv(
    '../data/raw/flight.csv', 
    sep=';',
    index=False)

In [28]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.
